In [19]:
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt
from matplotlib.pyplot import show
import numpy as np
from skimage.feature import hog
import joblib

Feature extraction functions

In [20]:
def remove_scrollbar(image, bar_width = 20):
    # gets rid of scroll bar so that model wont learn from that and returns the image
    y,x,_ = image.shape
    img = image[0:y,0:x-bar_width]
    return img


def extract_features(resized_image, orientations=10, pixels_per_cell=(6,6), cells_per_block=(1, 1)):
    # applied on resized image
    fd, hog_image = hog(resized_image, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True, channel_axis=-1)
    return fd, hog_image


def resize_image(image, new_size = (256, 128)): # size to resize to after future extraction
    return cv.resize(image, new_size)


def resize_and_get_feature_vector(image):
    # returns 1d feature vector to be appended to df
    resized_image = resize_image(image)
    fd, hog_image = extract_features(resized_image)

    return fd

Dataset preperation functions

In [21]:
def get_fieldnames(n_features = 8820): # with image of size (256, 128) 
    fieldnames = ['file_name', 'label']
    feature_values =  ['x_'+str(i) for i in range(n_features)]
    fieldnames += feature_values

    return fieldnames

def get_row(file_name, label, fd):
    new_row = {'file_name': file_name, 'label': label}
    for i,px_val in enumerate(fd.flatten()):
        new_row['x_'+str(i)] = px_val

    return new_row

In [22]:
model_path = "../models/svm_best_pipeline.joblib"
classifier = joblib.load(model_path)
classifier

Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection',
                 PCA(n_components=150, svd_solver='randomized', whiten=True)),
                ('classifier',
                 SVC(C=25425.16650996527, class_weight='balanced',
                     gamma=0.003479908281039559))])

Inference

In [23]:
def infer_image(image_path):
    image = cv.imread(image_path)

    # feature extraction
    fd = resize_and_get_feature_vector(image)
    # 2d array is expected
    prediction = classifier.predict(fd.reshape(1,-1))

    return prediction[0]

In [26]:
file_path = '../test_screenshots/Screenshot (21).png'

inference = infer_image(file_path)
# the warning comes from the fact that the model is initially trained on a dataset while prediction is made on single value. can be ignored
print('Inferred website: ', inference)

Inferred website:  amazon


c:\Users\Hakan Ergin\Desktop\code\website-classification\env\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
